In [8]:
import pandas as pd
import numpy as np

In [9]:
# Crear el DataFrame original con las probabilidades
# Estos datos se obtienen de una tabla de probabilidades de goles en un partido de fútbol hecha con distribución de Poisson
data = {
    'Goles': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'Prob_Local': [6.21, 17.26, 23.98, 22.21, 15.43, 8.57, 3.97, 1.58, 0.55],
    'Prob_Visitante': [49.33, 34.86, 12.32, 2.9, 0.51, 0.07, 0.01, 0.0, 0.0]
}

df_original = pd.DataFrame(data)

# Crear el DataFrame multiplicado
df_multiplicado = pd.DataFrame(index=df_original['Goles'], columns=df_original['Goles'])

# Realizar la multiplicación de probabilidades para cada posible resultado
for i in range(len(df_original)):
    for j in range(len(df_original)):
        goles_local = df_original['Goles'][i]
        goles_visitante = df_original['Goles'][j]
        probabilidad_conjunta = df_original['Prob_Local'][i] * df_original['Prob_Visitante'][j]
        df_multiplicado.loc[goles_local, goles_visitante] = (probabilidad_conjunta / 100)

# Transponer el DataFrame para tener el eje horizontal como goles del equipo local
df_multiplicado = df_multiplicado.transpose()

In [10]:
new_df = pd.DataFrame(data=df_multiplicado.values)
new_df

,0,1,2,3,4,5,6,7,8
0,3.063393,8.514358,11.829334,10.956193,7.611619,4.227581,1.958401,0.779414,0.271315
1,2.164806,6.016836,8.359428,7.742406,5.378898,2.987502,1.383942,0.550788,0.19173
2,0.765072,2.126432,2.954336,2.736272,1.900976,1.055824,0.489104,0.194656,0.06776
3,0.18009,0.50054,0.69542,0.64409,0.44747,0.24853,0.11513,0.04582,0.01595
4,0.031671,0.088026,0.122298,0.113271,0.078693,0.043707,0.020247,0.008058,0.002805
5,0.004347,0.012082,0.016786,0.015547,0.010801,0.005999,0.002779,0.001106,0.000385
6,0.000621,0.001726,0.002398,0.002221,0.001543,0.000857,0.000397,0.000158,0.000055
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Sumar la mitad superior de la tabla (por encima de la diagonal principal)
suma_mitad_superior = df_multiplicado.where(np.triu(np.ones(df_multiplicado.shape), k=1).astype(bool)).sum().sum()

# Sumar la mitad inferior de la tabla (por debajo de la diagonal principal)
suma_mitad_inferior = df_multiplicado.where(np.tril(np.ones(df_multiplicado.shape), k=-1).astype(bool)).sum().sum()

# Sumar la diagonal principal de la tabla original
suma_diagonal_principal = df_multiplicado.values.diagonal().sum()

print("Probabilidades de Resultados de Partido")
print(f"Gana local: {suma_mitad_superior:.2f}%")
print(f"Gana visitante: {suma_mitad_inferior:.2f}%")
print(f"Empate: {suma_diagonal_principal:.2f}%")

print("\nProbabilidades implícitas")
print(f"Local: {(1 / suma_mitad_superior)*100:.4f}")
print(f"Visitante: {(1 / suma_mitad_inferior)*100:.4f}")
print(f"Empate: {(1 / suma_diagonal_principal)*100:.4f}")

Probabilidades de Resultados de Partido
Gana local: 80.14%
Gana visitante: 6.86%
Empate: 12.76%

Probabilidades implícitas
Local: 1.2478
Visitante: 14.5846
Empate: 7.8347


## Probabilidades de total de goles

In [12]:
# Posibilidad de que el partido tenga menos de 0.5 goles (0-0)
less_05 = df_multiplicado.loc[0, 0]

# Posibilidad de que el partido tenga menos de 1.5 goles
less_15 = df_multiplicado.loc[0, 0] + df_multiplicado.loc[1, 0] + df_multiplicado.loc[0, 1]

# Posibilidad de que el partido tenga menos de 2.5 goles
less_25 = df_multiplicado.loc[0, 0] + df_multiplicado.loc[1, 0] + df_multiplicado.loc[0, 1] + df_multiplicado.loc[1, 1] + df_multiplicado.loc[2, 0] + df_multiplicado.loc[0, 2]

# Posibilidad de que el partido tenga menos de 3.5 goles
less_35 = df_multiplicado.loc[0, 0] + df_multiplicado.loc[1, 0] + df_multiplicado.loc[0, 1] + df_multiplicado.loc[1, 1] + df_multiplicado.loc[2, 0] + df_multiplicado.loc[0, 2] + df_multiplicado.loc[1, 2] + df_multiplicado.loc[2, 1] + df_multiplicado.loc[0, 3] + df_multiplicado.loc[3, 0]

# Posibilidad de que el partido tenga menos de 4.5 goles
less_45 = df_multiplicado.loc[0, 0] + df_multiplicado.loc[1, 0] + df_multiplicado.loc[0, 1] + df_multiplicado.loc[1, 1] + df_multiplicado.loc[2, 0] + df_multiplicado.loc[0, 2]  + df_multiplicado.loc[1, 2] + df_multiplicado.loc[2, 1] + df_multiplicado.loc[0, 3] + df_multiplicado.loc[3, 0] + df_multiplicado.loc[3, 1] + df_multiplicado.loc[1, 3] + df_multiplicado.loc[2, 2] + df_multiplicado.loc[4, 0] + df_multiplicado.loc[0, 4]

print("Probabilidades de goles")
print(f"Menos de 0.5 goles: {less_05:.2f}%")
print(f"Menos de 1.5 goles: {less_15:.2f}%")
print(f"Menos de 2.5 goles: {less_25:.2f}%")
print(f"Menos de 3.5 goles: {less_35:.2f}%")
print(f"Menos de 4.5 goles: {less_45:.2f}%")

print("-------------------------------------")

print(f"Más de 0.5 goles: {100 - less_05:.2f}%")
print(f"Más de 1.5 goles: {100 - less_15:.2f}%")
print(f"Más de 2.5 goles: {100 - less_25:.2f}%")
print(f"Más de 3.5 goles: {100 - less_35:.2f}%")
print(f"Más de 4.5 goles: {100 - less_45:.2f}%")

print("\nProbabilidades implícitas")
print(f"Menos de 0.5 goles: {(1 / less_05)*100:.2f}")
print(f"Menos de 1.5 goles: {(1 / less_15)*100:.2f}")
print(f"Menos de 2.5 goles: {(1 / less_25)*100:.2f}")
print(f"Menos de 3.5 goles: {(1 / less_35)*100:.2f}")
print(f"Menos de 4.5 goles: {(1 / less_45)*100:.2f}")

print("-------------------------------------")
print(f"Más de 0.5 goles: {(1 / (100 - less_05))*100:.2f}")
print(f"Más de 1.5 goles: {(1 / (100 - less_15))*100:.2f}")
print(f"Más de 2.5 goles: {(1 / (100 - less_25))*100:.2f}")
print(f"Más de 3.5 goles: {(1 / (100 - less_35))*100:.2f}")
print(f"Más de 4.5 goles: {(1 / (100 - less_45))*100:.2f}")


Probabilidades de goles
Menos de 0.5 goles: 3.06%
Menos de 1.5 goles: 13.74%
Menos de 2.5 goles: 32.35%
Menos de 3.5 goles: 53.98%
Menos de 4.5 goles: 72.82%
-------------------------------------
Más de 0.5 goles: 96.94%
Más de 1.5 goles: 86.26%
Más de 2.5 goles: 67.65%
Más de 3.5 goles: 46.02%
Más de 4.5 goles: 27.18%

Probabilidades implícitas
Menos de 0.5 goles: 32.64
Menos de 1.5 goles: 7.28
Menos de 2.5 goles: 3.09
Menos de 3.5 goles: 1.85
Menos de 4.5 goles: 1.37
-------------------------------------
Más de 0.5 goles: 1.03
Más de 1.5 goles: 1.16
Más de 2.5 goles: 1.48
Más de 3.5 goles: 2.17
Más de 4.5 goles: 3.68


### Probabilidades de Resultado Exacto 

In [13]:
df_r1 = pd.DataFrame(df_multiplicado)

# Asumiendo que ya tienes tu DataFrame df_r1

# Seleccionar las filas y columnas relevantes (sin contar hasta [3, 3])
cuadro_prin = df_r1.loc['0':'3', '0':'3']

# Sumar todas las probabilidades de la parte superior de la diagonal
probabilidad_superior_diagonal = cuadro_prin.where(np.triu(np.ones(cuadro_prin.shape), k=1).astype(bool)).sum().sum()

# Sumar la mitad inferior de la tabla (por debajo de la diagonal principal)
probabilidad_inferior_diagonal = cuadro_prin.where(np.tril(np.ones(cuadro_prin.shape), k=-1).astype(bool)).sum().sum()

# Sumar la diagonal principal de la tabla original
prob_diagonal_principal = cuadro_prin.values.diagonal().sum()

# Calcular las probabilidades implícitas
rLocal = suma_mitad_superior - probabilidad_superior_diagonal
rVisitante = suma_mitad_inferior - probabilidad_inferior_diagonal
rEmpate = suma_diagonal_principal - prob_diagonal_principal


In [14]:
rows = ["0 - 0", "0 - 1", "0 - 2", "0 - 3", "1 - 0", "1 - 1", "1 - 2", "1 - 3", "2 - 0", "2 - 1", "2 - 2", "2 - 3", "3 - 0", "3 - 1", "3 - 2", "3 - 3", "Otro, victoria local", "Otro, victoria visitante", "Otro, empate"]

data2 = {
    'Probabilidad': [df_multiplicado.loc[0, 0]
                     , df_multiplicado.loc[1, 0]
                    , df_multiplicado.loc[2, 0]
                    , df_multiplicado.loc[3, 0]
                    , df_multiplicado.loc[0, 1]
                    , df_multiplicado.loc[1, 1]
                    , df_multiplicado.loc[2, 1]
                    , df_multiplicado.loc[3, 1]
                    , df_multiplicado.loc[0, 2]
                    , df_multiplicado.loc[1, 2]
                    , df_multiplicado.loc[2, 2]
                    , df_multiplicado.loc[3, 2]
                    , df_multiplicado.loc[0, 3]
                    , df_multiplicado.loc[1, 3]
                    , df_multiplicado.loc[2, 3]
                    , df_multiplicado.loc[3, 3]
                    ,probabilidad_superior_diagonal
                    ,probabilidad_inferior_diagonal
                    ,prob_diagonal_principal
                    ],
    'Implícito': [100 / df_multiplicado.loc[0, 0], 
                  100 / df_multiplicado.loc[1, 0],
                  100 / df_multiplicado.loc[2, 0],
                  100 / df_multiplicado.loc[3, 0],
                  100 / df_multiplicado.loc[0, 1],
                  100 / df_multiplicado.loc[1, 1],
                  100 / df_multiplicado.loc[2, 1],
                  100 / df_multiplicado.loc[3, 1],
                  100 / df_multiplicado.loc[0, 2],
                  100 / df_multiplicado.loc[1, 2],
                  100 / df_multiplicado.loc[2, 2],
                  100 / df_multiplicado.loc[3, 2],
                  100 / df_multiplicado.loc[0, 3],
                  100 / df_multiplicado.loc[1, 3],
                  100 / df_multiplicado.loc[2, 3],
                  100 / df_multiplicado.loc[3, 3],
                  100 / rLocal, 
                  100 / rVisitante,
                  100 / rEmpate
                  ]
}

# Create the new dataframe
df_result_exac = pd.DataFrame(columns=["Probabilidad (%)", "Implícito"], index=rows)

df_result_exac["Probabilidad (%)"] = data2["Probabilidad"]
df_result_exac["Implícito"] = data2["Implícito"]

df_result_exac


,Probabilidad (%),Implícito
0 - 0,3.063393,32.643543
0 - 1,2.164806,46.193516
0 - 2,0.765072,130.706652
0 - 3,0.180090,555.277917
1 - 0,8.514358,11.744867
1 - 1,6.016836,16.620031
1 - 2,2.126432,47.027133
1 - 3,0.500540,199.784233
2 - 0,11.829334,8.453561
2 - 1,8.359428,11.962541
